In [1]:
import json
import time
import random
import dask.bag as db
import bs4
import requests
from fake_useragent import UserAgent
from tqdm.notebook import tqdm
from functions.scrape import get_business_data

import warnings # Turn off warnings
warnings.filterwarnings('ignore')

In [ ]:
save=True

# Set Params

In [2]:
with open(r'data/urls.txt') as f:
    urls = f.read().splitlines()
    
urls[0]

'https://www.yelp.com/biz/aesthetic-climbing-gym-lake-forest'

# Get Business Data

In [3]:
# Warning! Monitor the loop for Captchas

data_json = [] # Collect results

for url in tqdm(urls):
    
    time.sleep(10+10*random.random())
    
    ua = UserAgent() # Generate random user to make it seem like the task is distributed across multiple computers
    user_agent = {'User-agent': ua.random}
    response  = requests.get(url, headers = user_agent)
    soup = bs4.BeautifulSoup(response.text)    
    
    # Print information for tracking
#     page_text = soup.find('div',{"class":"page-of-pages arrange_unit arrange_unit--fill"}).get_text().strip() # Get 'Page <current> of <total>'
#     num_pages = int(re.search('(?<=of) [0-9]*', page_text).group(0).strip()) # Get total number of pages
#     num_reviews = soup.find('span',{"class":"review-count rating-qualifier"}).get_text().strip() # Get total number of reviews
#     business_id = soup.find("div", {"class": "lightbox-map hidden"}).get("data-business-id")
#     print(*[url, business_id, 'Index: '+str(url_index), num_reviews], sep="\n") # Quality Control
        
    data_dict = get_business_data(soup)
    data_json.append(data_dict) # Collect data

In [4]:
if save:
    filename = "business"
    with open(f'data/{filename}.json', 'w') as f:
        for review in tqdm(data_json):
            json.dump(review, f)
            f.write('\n')
    print('Complete')

Complete


# Check

In [5]:
business_full_bag = db.read_text("data/business.json").map(json.loads) # Loads the json file as a dask bag
business_tuple = business_full_bag.take(10000) # Takes the first 10000 entries of the dask bag and stores as a tuple
len(business_tuple)

71